<a href="https://colab.research.google.com/github/Tangkasbarap/tangkas-sps/blob/main/5_Sinyal_Tematik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**PROGRAM DARI SINYAL ASLI TEMATIK**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, Dropdown, FloatSlider, interactive
from scipy.fft import fft, fftfreq
from scipy.linalg import dft
import ipywidgets as widgets
from IPython.display import display

###FUNGSI UNTUK MEMUNCULKAN SINYAL SENSOR

In [ ]:
##Model Matematis##

def sensor_pressure(freq=1, amplitude=10):
    sampling_rate = 1000
    offset = 1013
    t = np.linspace(0,1, sampling_rate)
    P = amplitude * np.sin(2 * np.pi * freq * t)+ offset
    return t, P


def sensor_seismic(freq=15, amplitude=0.11 * 9.8):
    durasi = 3
    sampling_rate = 1000
    t = np.arange(0, durasi, 1 / sampling_rate)
    sinyal_sinus = amplitude * np.sin(2 * np.pi * freq * t)
    return t, sinyal_sinus

def sensor_temperature(freq=1, amplitude=1, sampling_rate=1000, duration=1, base_temp=25, signal_freq=5, signal_amp=2):
    sampling_rate = 1000
    duration = 1
    time = np.linspace(0, duration, int(sampling_rate * duration) )
    t = np.linspace(0, duration, int(sampling_rate * duration), endpoint=False)
    T = base_temp + signal_amp * np.sin(2 * np.pi * freq * t)
    sinyal_temp = amplitude * T
    return t, sinyal_temp

def sensor_asap(freq=5, amplitude=5.0):
    durasi = 1
    sampling_rate = 1000
    t = np.linspace(0, durasi, int(sampling_rate * durasi), endpoint=False)
    sinyal_asap = amplitude * np.sin(2 * np.pi * freq * t)
    return t, sinyal_asap


def sensor_gas(freq=10.0, amplitude=5.0, R0=20000, RL=10000, a=1.5, b=-0.6, resistance_model='linear'):
    t = np.linspace(0, 1, 1000, endpoint=False)  # Waktu
    C_gas = amplitude * np.sin(2 * np.pi * freq * t) + amplitude
    C_gas = np.clip(C_gas, 1, None)  # Konsentrasi tidak boleh negatif
    Rs = R0 * a * C_gas**b if resistance_model == 'linear' else R0 * np.exp(a * C_gas**b)
    Vout = (5.0 * 10000) / (10000 + Rs)
    return t, Vout

def add_noise(t, signal, noise_freq, noise_amplitude, sensor_type='default', resistance_model='linear'):
    if sensor_type == "seismic":
        noise = noise_amplitude * np.sin(2 * np.pi * noise_freq * t)
        return signal + noise
    elif sensor_type == "tgs":
        noise = noise_amplitude * np.sin(2 * np.pi * noise_freq * t)
        noisy_concentration = signal + noise
        noisy_concentration = np.clip(noisy_concentration, 1, None)
        if resistance_model == 'linear':
            noisy_resistance = 20000 * 1.5 * noisy_concentration**-0.6
        else:
            noisy_resistance = 20000 * np.exp(1.5 * noisy_concentration**-0.6)
        return noisy_concentration
    else:
        noise = noise_freq * np.sin(2 * np.pi * noise_amplitude * t)
        return signal + noise

##FUNGSI UNTUK ANALISIS SPEKTRUM DARI SINYAL SENSOR

In [ ]:
def spektrum(signal, sampling_rate, title="Spektrum Frekuensi"):
    N = len(signal)
    freq = np.fft.fftfreq(N, d=1/sampling_rate)
    DFT_signal = np.fft.fft(signal)
    return freq[:N//2], np.abs(DFT_signal[:N//2])


##FUNGSI UNTUK PEMILIHAN SENSOR,PLOTTING,NOISE

In [ ]:
def interactive_analysis(sensor_name, freq=10, amplitude=1, noise_freq=10, noise_amplitude=10, tegangan=5):
    # Pilih sensor
    base_concentration = 10
    resistance_model = 'linear'
    if sensor_name == "Pressure BMP180":
        t, signal = sensor_pressure(freq=freq, amplitude=amplitude)
    elif sensor_name == "Seismic ADXL345":
        t, signal = sensor_seismic(freq, amplitude)
        signal_with_noise = add_noise(t, signal, noise_freq, noise_amplitude, sensor_type='seismic')  # Tentukan sensor_type
    elif sensor_name == "Temperature DHT22":
        t, signal = sensor_temperature(freq, tegangan)
    elif sensor_name == "Asap MQ2":
        t, signal = sensor_asap(freq, tegangan )
    elif sensor_name == "Gas TGS2600":
        t, signal = sensor_gas(freq, amplitude,base_concentration, resistance_model)
        signal_with_noise = add_noise(t, signal, noise_freq, noise_amplitude, sensor_type='tgs')  # Tentukan sensor_type

    plt.figure(figsize=(12, 8))
    ##Label Y
    if sensor_name == "Temperature DHT22":
        plt.plot(t, signal, label=f"Sinyal {sensor_name}", color="Blue")  # Menggunakan signal, bukan signal * tegangan
        plt.ylabel("Temperatur (°C)")
    elif sensor_name == "Pressure BMP180":
        plt.plot(t, signal * tegangan, label=f"Sinyal {sensor_name}", color="Blue")
        plt.ylabel("Tekanan (P)")
    elif sensor_name == "Seismic ADXL345":
        plt.plot(t, signal * tegangan, label=f"Sinyal {sensor_name}", color="Blue")
        plt.ylabel("Akselerasi (G)")
    elif sensor_name == "Asap MQ2":
        plt.plot(t, signal * tegangan, label=f"Sinyal {sensor_name}", color="Blue")
        plt.ylabel("Smoke ")
    elif sensor_name == "Gas TGS2600":
        plt.plot(t, signal * tegangan, label=f"Sinyal {sensor_name}", color="Blue")
        plt.ylabel("PPM (%)")

    plt.title(f"Sinyal Asli - {sensor_name}")
    plt.xlabel("Waktu (s)")
    plt.legend()
    plt.grid()
    plt.show()

    # Tambahkan Noise
    if sensor_name == "Gas TGS2600":
        signal_with_noise = add_noise(t, signal, noise_freq, noise_amplitude, sensor_type='tgs', resistance_model=resistance_model)
    else: signal_with_noise = add_noise(t, signal, noise_freq, tegangan)



    # Spektrum Awal
    sampling_rate = 1000
    if sensor_name == "Temperature DHT22":
        freq, dft = spektrum(signal*tegangan, sampling_rate)
    elif sensor_name == "Pressure BMP180":
        freq, dft = spektrum(signal * tegangan ,sampling_rate)
    elif sensor_name == "Seismic ADXL345":
        freq, dft = spektrum(signal  ,sampling_rate)
    elif sensor_name == "Asap MQ2":
        freq, dft = spektrum(signal , sampling_rate)
    elif sensor_name == "Gas TGS2600":
        freq, dft = spektrum(signal , sampling_rate)
    plt.figure(figsize=(12, 8))
    plt.plot(freq, np.abs(dft))
    plt.title(f"DFT(Magnitude Spectrum) of Sensor - {sensor_name}")
    plt.xlabel("Frekuensi (Hz)")
    plt.ylabel("Amplitudo")
    plt.xlim(0, 100)
    plt.grid()
    plt.show()
    # spektrum(t, signal, title=f"Spektrum Awal - {sensor_name}" )
    # Plot Sinyal dengan Gangguan
    plt.figure(figsize=(12, 8))
    plt.plot(t, signal_with_noise, label=f"Sinyal dengan Gangguan - {sensor_name}", color="Orange")
    plt.title(f"Sinyal dengan Gangguan {noise_freq} Hz + {sensor_name}")
    plt.xlabel("Waktu (s)")
    plt.ylabel("Amplitude")
    plt.legend()
    plt.grid()
    plt.show()




# **GUI**

In [ ]:
def create_gui():
    # Dropdown untuk memilih sensor
    sensor_dropdown = widgets.Dropdown(
        options=["Asap MQ2", "Seismic ADXL345", "Temperature DHT22", "Pressure BMP180", "Gas TGS2600"],
        description="Sensor:"
    )

    # Slider untuk frekuensi asli
    freq_slider = widgets.FloatSlider(
        value=10, min=0.1, max=100, step=0.1, description="Asli (Hz):"
    )

    # Slider untuk tegangan
    tegangan_slider = widgets.FloatSlider(
        value=5, min=0.1, max=10, step=0.1, description="Tegangan:"
    )

    # Slider untuk frekuensi noise
    noise_freq_slider = widgets.FloatSlider(
        value=50, min=0.1, max=100, step=0.1, description="Noise (Hz):"
    )

    # Slider untuk amplitudo noise
    noise_amplitude_slider = widgets.FloatSlider(
        value=10, min=0.1, max=5.0, step=0.1, description="Amp Noise:"
    )

    # Tombol untuk menjalankan analisis
    analyze_button = widgets.Button(description="Jalankan Analisis")

    # Output untuk menampilkan hasil analisis
    output = widgets.Output()

    # Fungsi untuk dijalankan saat tombol ditekan
    def on_button_clicked(b):
        with output:
            output.clear_output()
            interactive_analysis(
                  sensor_dropdown.value,
                  freq_slider.value,
                  noise_freq_slider.value,
                  noise_amplitude_slider.value,
                  tegangan_slider.value
            )

    # Menambahkan event listener ke tombol
    analyze_button.on_click(on_button_clicked)

    # Menampilkan GUI
    # Moved the display call inside the create_gui function
    display(
        widgets.VBox([
            sensor_dropdown,
            freq_slider,
            tegangan_slider,
            noise_freq_slider,
            noise_amplitude_slider,
            analyze_button,
            output
        ])
    )

# Memanggil fungsi untuk membuat GUI
create_gui()

## SLIDER UNTUK PROGRAM INTERAKTIFNYA

In [ ]:
interact(
    interactive_analysis,
    sensor_name=Dropdown(options=["Asap MQ2", "Seismic ADXL345", "Temperature DHT22","Pressure BMP180","Gas TGS2600"], description="Sensor"),
    freq=FloatSlider(value=10, min=0.1, max=100, step=0.1, description="Asli (Hz)"),
    #amplitude=FloatSlider(value=10, min=0.1, max=10, step=0.1, description="Amp Asli"),
    tegangan=FloatSlider(value=5, min=0.1, max=10, step=0.1, description="Tegangan"),
    noise_freq=FloatSlider(value=50, min=0.1, max=100, step=0.1, description="Noise (Hz)"),
    noise_amplitude=FloatSlider(value=10, min=0.1, max=5.0, step=0.1, description="Amp Noise"),
)


interactive(children=(Dropdown(description='Sensor', options=('Asap MQ2', 'Seismic ADXL345', 'Temperature DHT2…

<function __main__.interactive_analysis(sensor_name, freq=10, amplitude=1, noise_freq=10, noise_amplitude=10, tegangan=5)>